## お笑い（特に漫才）のテキストデータ分析

### 問い

1. 関西と関東で言葉の言い回しが違う？
    1. 関西は単純な言葉とノリで、関東はひねった言い回し？
2. 各芸人のネタを共起ネットワークに起こしてみると、芸人によって何か特徴が見られるか
3. ...

漫才のデータ分析をした記事もあったので参考にする。
感情分析はちょっと面白そう
- [データ分析で、M-1グランプリの"高得点パターン"を探してみた](https://note.com/natto_nebaneba/n/n11b4229f8b19)


### データの用意

ネタ台本の書き起こしをしているサイトからとってくる

量が多くてデータを用意しやすそう
- https://himapucchi.com/
- https://comedy.shivatax.com/category/manzaiwrite/
- https://waraitext.com/category/neta/manzai/

必要だったら追加で参照
- https://manzaidaihon.com/
- https://bukiyo30.hateblo.jp/
- https://hansode5.com/category/comedy-english/manzai/




# Code

### スクレイピング

https://himapucchi.com/
のスクレイピング

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# フォルダ
folder_name_raw = './data/raw'
print(os.path.exists(folder_name_raw))

folder_name_tidy = "./data/tidy"
print(os.path.exists(folder_name_tidy))

True
True


In [3]:
root_url = "https://himapucchi.com/"

In [4]:
# 17ページに分かれているのでlistを作成

crawl_list = []
crawl_list.append(root_url)

for i in range(16):
    crawl_list.append(f"{root_url}page/{i+2}")

crawl_list

['https://himapucchi.com/',
 'https://himapucchi.com/page/2',
 'https://himapucchi.com/page/3',
 'https://himapucchi.com/page/4',
 'https://himapucchi.com/page/5',
 'https://himapucchi.com/page/6',
 'https://himapucchi.com/page/7',
 'https://himapucchi.com/page/8',
 'https://himapucchi.com/page/9',
 'https://himapucchi.com/page/10',
 'https://himapucchi.com/page/11',
 'https://himapucchi.com/page/12',
 'https://himapucchi.com/page/13',
 'https://himapucchi.com/page/14',
 'https://himapucchi.com/page/15',
 'https://himapucchi.com/page/16',
 'https://himapucchi.com/page/17']

In [23]:
# <a>のlistを引数に、<a>内のURLとtitleを取得しdict listを返す
def extract_url_title_himapucchi(a_list):
    page_url_title = []
    for content in a_list:
        url = content.get('href')  # href属性を取得
        title = content.get("title")  # titleを取得
        page_url_title.append({"page_url": url, "title": title})
    return page_url_title


In [24]:
# User-Agent の設定
dummy_user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'


In [27]:
# htmlリクエスト & soup

page_url_title = []

for crawl_url in crawl_list:
    response = requests.get(crawl_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")

    page_list = soup.select("#main > #list > a")
    page_url_title.extend(extract_url_title_himapucchi(page_list))

    time.sleep(1) # 負荷軽減


In [28]:
page_df = pd.DataFrame(page_url_title)
page_df

,page_url,title
0,https://himapucchi.com/m2023-final-kurage/,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」
1,https://himapucchi.com/m2023-final-danbira/,【M-1グランプリ2023 1stラウンド】ダンビラムーチョ「カラオケボックス」
2,https://himapucchi.com/m2023-final-shinkujeshika/,【M-1グランプリ2023 1stラウンド】真空ジェシカ「B画館」
3,https://himapucchi.com/m2023-final-mayurika/,【M-1グランプリ2023 1stラウンド】マユリカ「倦怠期」
4,https://himapucchi.com/m2023-final-kabeposutar/,【M-1グランプリ2023 1stラウンド】カベポスター「おまじない」
...,...,...
163,https://himapucchi.com/m2019first-milkboy/,【M-1グランプリ2019 1stラウンド】<br>ミルクボーイ「コーンフレーク」
164,https://himapucchi.com/m2019first-kamaitachi/,【M-1グランプリ2019 1stラウンド】<br>かまいたち「UFJ」
165,https://himapucchi.com/m2019final-milkboy/,【M-1グランプリ2019 最終決戦】<br>ミルクボーイ「最中」
166,https://himapucchi.com/m2019final-kamaitachi/,【M-1グランプリ2019 最終決戦】<br>かまいたち「トトロ」


In [31]:
# いったんcsvに保存
page_df.to_csv(os.path.join(folder_name_raw, "page_url_title.csv"), index=False)

In [52]:
# <div>listからscriptの抽出
def extract_script_himapucchi(script_div_list, title):
    script_dict_list = []
    name_pos_r = "" # 右側の芸人の名前
    name_pos_l = "" # 左側の芸人の名前
    for i, script_div in enumerate(script_div_list):
        clas = script_div.get("class")
        text = script_div.select_one("p")
        if text:
            text = text.get_text(strip=True)
        name = script_div.select_one("div.speech-name")
        if name:
            name = name.get_text(strip=True)
            if "sbp-r" in clas:
                name_pos_r = name
                print("r: " + name_pos_r)
            elif "sbp-l" in clas:
                name_pos_l = name
                print("l: " + name_pos_l)
            else:
                raise "invalid class (name)"
        
        if "sbp-r" in clas: # 右側の芸人のセリフ
            script_dict_list.append({"title": title, "pos": "r", "name": name_pos_r, "turn": i, "speech": text})
        elif "sbp-l" in clas: # 左側の芸人のセリフ
            script_dict_list.append({"title": title, "pos": "l", "name": name_pos_l, "turn": i, "speech": text})
        else:
            raise "invalid class"
    
    return script_dict_list
            

# ページからtitleとscriptを抽出
def extract_title_script_himapucchi(page_url):
    response = requests.get(page_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.select_one(".entry-title").get_text(strip=True)
    print(title)

    script_div_list = soup.select(".entry-content > div.speech-wrap")
    script_dict_list = extract_script_himapucchi(script_div_list, title)

    return script_dict_list

    

In [53]:
# scriptのスクレイピング

script_df_all = pd.DataFrame()
for page_url in page_df["page_url"]:
    script_df = pd.DataFrame(extract_title_script_himapucchi(page_url))
    script_df_all = pd.concat([script_df_all, script_df], ignore_index=True)
    time.sleep(1) # 負荷軽減

script_df_all

【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」
r: 渡辺
l: 杉
【M-1グランプリ2023 1stラウンド】ダンビラムーチョ「カラオケボックス」
【M-1グランプリ2023 1stラウンド】真空ジェシカ「B画館」
r: 川北
l: ガク
【M-1グランプリ2023 1stラウンド】マユリカ「倦怠期」
r: 中谷
l: 阪本
【M-1グランプリ2023 1stラウンド】カベポスター「おまじない」
r: 浜田
l: 永見
【M-1グランプリ2023 1stラウンド】シシガシラ「規制」
r: 脇田
l: 浜中
【M-1グランプリ2023 最終決戦】さや香「見せ算」
r: 石井
l: 新山
【M-1グランプリ2023 最終決戦】ヤーレンズ「ラーメン屋」
r: 出井
l: 楢原
【M-1グランプリ2023 1stラウンド】ヤーレンズ「引越しの挨拶」
r: 出井
l: 楢原
【M-1グランプリ2023 1stラウンド】さや香「ホームステイ」
l: 新山
r: 石井
【M-1グランプリ2023 最終決戦】令和ロマン「ドラマ」
r: 松井
l: 髙比良
【M-1グランプリ2023 1stラウンド】令和ロマン「少女漫画」
r: 松井
l: 髙比良
【M-1グランプリ2023 3回戦】ヤーレンズ「ケーキ屋」
r: 出井
l: 楢原
【M-1グランプリ2023 3回戦】マユリカ「井戸端会議」
r: 中谷
l: 阪本
【M-1グランプリ2023 3回戦】令和ロマン「100m走」
r: 松井
l: 髙比良
【M-1グランプリ2023 3回戦】ナイチンゲールダンス「居酒屋」
r: ヤス
l: 中野
【M-1グランプリ2023 3回戦】くらげ「不幸」
l: 杉
r: 渡辺
【M-1グランプリ2023 3回戦】ダンビラムーチョ「昔話」
l: 大原
r: 原田
【M-1グランプリ2023 3回戦】エバース「略奪」
r: 町田
l: 佐々木
【M-1グランプリ2023 3回戦】シシガシラ「同窓会」
r: 脇田
l: 浜中
【M-1グランプリ2023 3回戦】ぎょうぶ「家族」
【M-1グランプリ2023 3回戦】ななまがり「マッチングアプリ」
【M-1グランプリ2023 3回戦】ママタルト「ラブコメ」
【M-1グランプリ2023 3回戦】さや香「大学」
l: 新山
r: 石井
【M-

,title,pos,name,turn,speech
0,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,0,ども！くらげです。お願いします
1,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,1,お願いします
2,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,2,すみません、ありがとうございます！頑張っていきましょうねー
3,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,3,あのー、俺さ、この前初めてさ、31アイスに行ったのよ
4,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,4,あ、31。でも31ってさ、店内カラフルだし女子高生とか多いからなんか、おじさんの俺達は入りづらいな
...,...,...,...,...,...
24680,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,89,ちょっとちょっと、ちょっと待ってくれ相方！
24681,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,90,え？
24682,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,91,ずっと何やってんだよ！
24683,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,92,いい加減にしろ！


In [54]:
# csvに保存
script_df_all.to_csv(os.path.join(folder_name_raw, "script_all.csv"), index=False)

### スクレイピング
https://comedy.shivatax.com/category/manzaiwrite/ のスクレイピング

In [8]:
# User-Agent の設定
dummy_user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'


In [6]:
root_url = "https://comedy.shivatax.com/category/manzaiwrite/"

In [7]:
# 11ページに分かれているのでlistを作成

crawl_list = []
crawl_list.append(root_url)

for i in range(10):
    crawl_list.append(f"{root_url}page/{i+2}")

crawl_list

['https://comedy.shivatax.com/category/manzaiwrite/',
 'https://comedy.shivatax.com/category/manzaiwrite/page/2',
 'https://comedy.shivatax.com/category/manzaiwrite/page/3',
 'https://comedy.shivatax.com/category/manzaiwrite/page/4',
 'https://comedy.shivatax.com/category/manzaiwrite/page/5',
 'https://comedy.shivatax.com/category/manzaiwrite/page/6',
 'https://comedy.shivatax.com/category/manzaiwrite/page/7',
 'https://comedy.shivatax.com/category/manzaiwrite/page/8',
 'https://comedy.shivatax.com/category/manzaiwrite/page/9',
 'https://comedy.shivatax.com/category/manzaiwrite/page/10',
 'https://comedy.shivatax.com/category/manzaiwrite/page/11']

In [10]:
# crawl_listから 各記事へのURL listを作成

script_url_list = []
for crawl_url in crawl_list:
    request = requests.get(crawl_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    articles = soup.select("#contentInner > main > article > aside > div.kanren > dl")
    for article in articles:
        title = article.select_one("dd > h3 > a").get_text(strip=True)
        link = article.select_one("dd > h3 > a").get("href")
        script_url_list.append({"page_url": link, "title": title})
    time.sleep(3)

url_title_df = pd.DataFrame(script_url_list)

url_title_df

,page_url,title
0,https://comedy.shivatax.com/linear-mens-idol/,リニア「メンズアイドル」
1,https://comedy.shivatax.com/oswald-long-wallet/,オズワルド「長財布」
2,https://comedy.shivatax.com/moglider-kenichimi...,モグライダー「美川憲一」
3,https://comedy.shivatax.com/wagyuu-charismatic...,和牛「カリスマ美容師」
4,https://comedy.shivatax.com/chidori-sushi-bar/,千鳥 「寿司屋」
...,...,...
98,https://comedy.shivatax.com/yonsentoushin-2/,四千頭身「好きな季節」
99,https://comedy.shivatax.com/footballhour/,フットボールアワー「SMタクシー」
100,https://comedy.shivatax.com/yonsentoushin/,四千頭身「ドライブ」
101,https://comedy.shivatax.com/ginsyari/,銀シャリ「ウンチク」


In [11]:
# csvに保存

url_title_df.to_csv(os.path.join(folder_name_raw, "page_url_title_shiva.csv"), index=False)

In [23]:
# shivataxの漫才台本を抽出する

script_dict_list = []
for index, page in url_title_df.iterrows():
    page_url = page["page_url"]
    title = page["title"]
    print(title, page_url)
    request = requests.get(page_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    speeches = soup.select("#nocopy > div > p")
    for i, speech in enumerate(speeches):
        speech_text = speech.get_text(strip=True)
        script_dict_list.append({"title": title, "turn": i, "speech": speech_text})
    time.sleep(3)

script_df_shiva = pd.DataFrame(script_dict_list)
script_df_shiva


リニア「メンズアイドル」 https://comedy.shivatax.com/linear-mens-idol/
オズワルド「長財布」 https://comedy.shivatax.com/oswald-long-wallet/
モグライダー「美川憲一」 https://comedy.shivatax.com/moglider-kenichimikawa/
和牛「カリスマ美容師」 https://comedy.shivatax.com/wagyuu-charismatic-hairdresser/
千鳥 「寿司屋」 https://comedy.shivatax.com/chidori-sushi-bar/
かまいたち「転校」 https://comedy.shivatax.com/kamaitachi-transferring/
かまいたち「怖い体験」 https://comedy.shivatax.com/kamaitachi-scary-experience/
千鳥 「飼育員」 https://comedy.shivatax.com/chidori-breeder/
NONSTYLE「時代劇」 https://comedy.shivatax.com/historical-play/
NON STYLE「ヤンキー漫画」 https://comedy.shivatax.com/nonstyle-yankee-manga/
フットボールアワー「フットボールアワー戦隊」 https://comedy.shivatax.com/football-hour-squadron/
フットボールアワー「結婚記者会見」 https://comedy.shivatax.com/football-hour-wedding-pressconference/
メイプル超合金「犬を飼いたい」 https://comedy.shivatax.com/maple-have-a-dog/
和牛「結婚式を抜け出すヒロイン」 https://comedy.shivatax.com/get-out-wedding/
和牛「ゾンビ」 https://comedy.shivatax.com/zombie/
タイムキーパー「幼稚園の先生」 https://comedy.shivatax.com/tim

,title,turn,speech
0,リニア「メンズアイドル」,0,赤字は最もウケたポイントM-1見るならAmazonプライム
1,リニア「メンズアイドル」,1,どうもお願いします。サカイケイタとりょうへいでリニアと言います。頑張ります。お願いします。あ...
2,リニア「メンズアイドル」,2,いやーケイタさ
3,リニア「メンズアイドル」,3,はいはい
4,リニア「メンズアイドル」,4,メンズアイドルやらない？
...,...,...,...
11146,ミルクボーイ「コーンフレーク」,93,オトンが言うにはな
11147,ミルクボーイ「コーンフレーク」,94,オトン？
11148,ミルクボーイ「コーンフレーク」,95,鯖の塩焼きちゃうかって言うて
11149,ミルクボーイ「コーンフレーク」,96,いや絶対ちゃうやろ。もうええわ


In [24]:
# csvに保存
script_df_shiva.to_csv(os.path.join(folder_name_raw, "script_all_shiva.csv"), index=False)

### スクレイピング
https://waraitext.com/category/neta/manzai/ のスクレイピング

In [12]:
root_url = "https://waraitext.com/category/neta/manzai/"

In [13]:
# 16ページに分かれているのでlistを作成

crawl_list = []
crawl_list.append(root_url)

for i in range(15):
    crawl_list.append(f"{root_url}page/{i+2}")

crawl_list

['https://waraitext.com/category/neta/manzai/',
 'https://waraitext.com/category/neta/manzai/page/2',
 'https://waraitext.com/category/neta/manzai/page/3',
 'https://waraitext.com/category/neta/manzai/page/4',
 'https://waraitext.com/category/neta/manzai/page/5',
 'https://waraitext.com/category/neta/manzai/page/6',
 'https://waraitext.com/category/neta/manzai/page/7',
 'https://waraitext.com/category/neta/manzai/page/8',
 'https://waraitext.com/category/neta/manzai/page/9',
 'https://waraitext.com/category/neta/manzai/page/10',
 'https://waraitext.com/category/neta/manzai/page/11',
 'https://waraitext.com/category/neta/manzai/page/12',
 'https://waraitext.com/category/neta/manzai/page/13',
 'https://waraitext.com/category/neta/manzai/page/14',
 'https://waraitext.com/category/neta/manzai/page/15',
 'https://waraitext.com/category/neta/manzai/page/16']

In [14]:
# crawl_listから 各記事へのURL listを作成

script_url_list = []
for crawl_url in crawl_list:
    print(crawl_url)
    request = requests.get(crawl_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    articles = soup.select("#contentInner > main > article > div.kanren > dl")
    for article in articles:
        title = article.select_one("dd > h3 > a").get_text(strip=True)
        link = article.select_one("dd > h3 > a").get("href")
        script_url_list.append({"page_url": link, "title": title})
    time.sleep(5) # 重そうなので長めに設定

url_title_df_warai = pd.DataFrame(script_url_list)

url_title_df_warai

https://waraitext.com/category/neta/manzai/
https://waraitext.com/category/neta/manzai/page/2
https://waraitext.com/category/neta/manzai/page/3
https://waraitext.com/category/neta/manzai/page/4
https://waraitext.com/category/neta/manzai/page/5
https://waraitext.com/category/neta/manzai/page/6
https://waraitext.com/category/neta/manzai/page/7
https://waraitext.com/category/neta/manzai/page/8
https://waraitext.com/category/neta/manzai/page/9
https://waraitext.com/category/neta/manzai/page/10
https://waraitext.com/category/neta/manzai/page/11
https://waraitext.com/category/neta/manzai/page/12
https://waraitext.com/category/neta/manzai/page/13
https://waraitext.com/category/neta/manzai/page/14
https://waraitext.com/category/neta/manzai/page/15
https://waraitext.com/category/neta/manzai/page/16


,page_url,title
0,https://waraitext.com/post-7791/,エバース【野球肘】
1,https://waraitext.com/post-7762/,ランジャタイ【谷村新司】
2,https://waraitext.com/post-7772/,モグライダー【マイケルじゃんけん】
3,https://waraitext.com/post-7680/,錦鯉【深夜の通販番組】
4,https://waraitext.com/post-7676/,TEAM BANANA【一周回って】
...,...,...
146,https://waraitext.com/post-92/,南海キャンディーズ【美容師】
147,https://waraitext.com/post-81/,スーパーマラドーナ【落ち武者】
148,https://waraitext.com/post-79/,海原千里万里【欧陽菲菲のショー】
149,https://waraitext.com/post-77/,B&B【岡山と広島】


In [15]:
# csvに保存

url_title_df_warai.to_csv(os.path.join(folder_name_raw, "page_url_title_warai.csv"), index=False)

In [27]:
# waraitextの漫才台本を抽出する

script_dict_list = []
for index, page in url_title_df_warai.iterrows():
    page_url = page["page_url"]
    title = page["title"]
    print(title, page_url)
    request = requests.get(page_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(request.text, "html.parser")
    speeches = soup.select("#nocopy > div > p")
    for i, speech in enumerate(speeches):
        speech_text = speech.get_text(strip=True)
        script_dict_list.append({"title": title, "turn": i, "speech": speech_text})
    time.sleep(5) # 重そうなので長めに設定

script_df_warai = pd.DataFrame(script_dict_list)
script_df_warai


エバース【野球肘】 https://waraitext.com/post-7791/
ランジャタイ【谷村新司】 https://waraitext.com/post-7762/
モグライダー【マイケルじゃんけん】 https://waraitext.com/post-7772/
錦鯉【深夜の通販番組】 https://waraitext.com/post-7680/
TEAM BANANA【一周回って】 https://waraitext.com/post-7676/
ぼる塾【モテたい】 https://waraitext.com/post-7607/
ダブルアート【グルメリポーター】 https://waraitext.com/post-7539/
風藤松原【やだね】 https://waraitext.com/post-7536/
真空ジェシカ【命の授業】 https://waraitext.com/post-7546/
ミルクボーイ【コーンフレーク】 https://waraitext.com/post-7565/
磁石【学生時代】 https://waraitext.com/post-7501/
ハリガネロック【見た目の誤解】 https://waraitext.com/post-7498/
ラランド【ビックな芸能人になりたい】 https://waraitext.com/post-7461/
令和ロマン【ナンパ】 https://waraitext.com/post-7451/
ヤーレンズ【ヒーローインタビュー】 https://waraitext.com/post-7443/
夢路いとし喜味こいし【昔は良かった】 https://waraitext.com/post-7426/
中田ダイマルラケット【健康法】 https://waraitext.com/post-7393/
オズワルド【買い物】 https://waraitext.com/post-6515/
ぺこぱ【プロポーズ】 https://waraitext.com/post-6511/
にぼしいわし【つけ麺】 https://waraitext.com/post-6517/
ミルクボーイ【サイゼ】 https://waraitext.com/post-6488/
エンペラー【女の子】 http

,title,turn,speech
0,エバース【野球肘】,0,
1,エバース【野球肘】,1,【関連】笑わせ方の根底にある「緊張の緩和」と「笑いの分類・サゲの分類」
2,エバース【野球肘】,2,【関連】お笑い ボケの種類 18種
3,エバース【野球肘】,3,【関連】笑いが起きる法則（フリ・オチ・フォロー）とは
4,エバース【野球肘】,4,
...,...,...,...
15812,紳助竜介【暴走族】,286,竜介「ふん」
15813,紳助竜介【暴走族】,287,紳助「かわいそうにあいつ死んでしまいよったー、ええ奴やったのにー、言うてんのはその日だけやで...
15814,紳助竜介【暴走族】,288,竜介「その日だけ」
15815,紳助竜介【暴走族】,289,紳助「３日もしてみい友達の家行って、あいつこないだ頭割れてなお前、なんぼ気色悪かったお前ー（...


In [28]:
# csvに保存
script_df_warai.to_csv(os.path.join(folder_name_raw, "script_all_warai.csv"), index=False)

### データの整形

#### himapucchiの整形

- titleが
```【[大会名]】[コンビ名]「[ネタタイトル]」```
となっているので、「コンビ名」と「ネタタイトル」を抽出（大会名はない場合もある）
- 一部「松本人志のすべらない話」「つかみネタまとめ」が入っているので削除



In [30]:
script_df_hima = pd.read_csv(os.path.join(folder_name_raw, "script_all.csv"))
script_df_hima

,title,pos,name,turn,speech
0,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,0,ども！くらげです。お願いします
1,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,1,お願いします
2,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,2,すみません、ありがとうございます！頑張っていきましょうねー
3,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,3,あのー、俺さ、この前初めてさ、31アイスに行ったのよ
4,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,4,あ、31。でも31ってさ、店内カラフルだし女子高生とか多いからなんか、おじさんの俺達は入りづらいな
...,...,...,...,...,...
24680,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,89,ちょっとちょっと、ちょっと待ってくれ相方！
24681,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,90,え？
24682,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,91,ずっと何やってんだよ！
24683,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,92,いい加減にしろ！


In [41]:
# 「人志松本のすべらない話」「つかみネタまとめ」を除去
script_df_hima_filtered = script_df_hima[~(script_df_hima["title"].str.contains("松本") | script_df_hima["title"].str.contains("つかみ"))]
script_df_hima_filtered


,title,pos,name,turn,speech
0,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,0,ども！くらげです。お願いします
1,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,1,お願いします
2,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,2,すみません、ありがとうございます！頑張っていきましょうねー
3,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,3,あのー、俺さ、この前初めてさ、31アイスに行ったのよ
4,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,4,あ、31。でも31ってさ、店内カラフルだし女子高生とか多いからなんか、おじさんの俺達は入りづらいな
...,...,...,...,...,...
24680,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,89,ちょっとちょっと、ちょっと待ってくれ相方！
24681,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,90,え？
24682,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,91,ずっと何やってんだよ！
24683,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,92,いい加減にしろ！


In [61]:
import re

In [68]:
# titleからコンビ名とネタタイトルを抽出

def extract_combi_neta_from_title(title):
    text = re.sub(r"【.+?】", "", title)
    pattern = r"(.+?)「(.+?)」"
    match_result = re.match(pattern, text)
    if not match_result:
        raise ValueError(f"invalid format: {title} ({text})")
    else:
        combi = match_result.group(1)
        neta = match_result.group(2)
        print(f"{combi}, {neta} extracted from {text}")
        return combi, neta


In [75]:
script_df_hima_titleParsed = script_df_hima_filtered.copy()

script_df_hima_titleParsed[["combi", "neta"]] = script_df_hima_titleParsed["title"].apply(extract_combi_neta_from_title).apply(pd.Series)
script_df_hima_titleParsed = script_df_hima_titleParsed[["combi", "neta", "turn", "speech"]]
script_df_hima_titleParsed

くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ extracted from くらげ「物忘れ」
くらげ, 物忘れ e

,combi,neta,turn,speech
0,くらげ,物忘れ,0,ども！くらげです。お願いします
1,くらげ,物忘れ,1,お願いします
2,くらげ,物忘れ,2,すみません、ありがとうございます！頑張っていきましょうねー
3,くらげ,物忘れ,3,あのー、俺さ、この前初めてさ、31アイスに行ったのよ
4,くらげ,物忘れ,4,あ、31。でも31ってさ、店内カラフルだし女子高生とか多いからなんか、おじさんの俺達は入りづらいな
...,...,...,...,...
24680,ぺこぱ,お年寄り,89,ちょっとちょっと、ちょっと待ってくれ相方！
24681,ぺこぱ,お年寄り,90,え？
24682,ぺこぱ,お年寄り,91,ずっと何やってんだよ！
24683,ぺこぱ,お年寄り,92,いい加減にしろ！


In [76]:
# csvに保存
script_df_hima_titleParsed.to_csv(os.path.join(folder_name_tidy, "script_hima.csv"), index=False)

In [79]:
# speechにNaNがあるが一旦スルー
script_df_hima_titleParsed[script_df_hima_titleParsed.isna().any(axis=1)]

,combi,neta,turn,speech
2004,シシガシラ,同窓会,60,NaN
2625,真空ジェシカ,バンパイアハンター,15,NaN
3033,ウエストランド,あるなしクイズ,7,NaN
8878,ナイツ,リオの戦リオ品,463,NaN
9771,ナイツ,ゴロゴロペー,35,NaN
13021,ナイツ,2010年を振り返りました,51,NaN
13154,ナイツ,SMAPをヤホーで調べました,36,NaN
13438,サンドウィッチマン,デリバリー,19,NaN
13996,サンドウィッチマン,テレビショッピング,5,NaN
14307,サンドウィッチマン,深夜のラジオ,74,NaN


#### shivatax の整形

- titleから「コンビ名」「ネタタイトル」を抽出
- 空白行を除去し、turnを詰める
- 先頭2行と最終行に台本でない文章（広告など）が入っていることがあるので除去

#### waraitext の整形

- titleから「コンビ名」「ネタタイトル」を抽出
- 空白行を除去し、turnを詰める
- 台本でない行を除去
- 芸人の名前と：や「」『』を除去
- （失礼ボケ）などの記述されたボケフリの種類を除去

### データの結合
3つの台本データを結合して一つのデータに

コラムは
- combi (コンビ名)
- neta (ネタタイトル)
- turn (発話ターンのカウント)
- speech (発話)